![question](img/hw5q5.png)

In [2]:
import numpy as np
import pandas as pd
import pyquantlib as pq
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload

In [2]:
# question assumptions
mat_list = [12, 18, 4*12]
cpn_rate_list = [0.05, 0, 0.08]
px_list = [103.0, 96.5, 116.0]
freq_list = [2, 2, 2]
init_rates = {0: 0.015}

In [3]:
# generating all of the cf times
t_lists = []
for i in range(0, len(mat_list)):
    cur_t_list = pq.bonds.gen_t_list(mat_list[i], freq_list[i])
    t_lists.append(cur_t_list)
t_lists

[array([ 0. ,  0.5,  1. ]),
 array([ 0. ,  0.5,  1. ,  1.5]),
 array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ])]

In [4]:
# generating all of the cf amounts
cf_lists = []
for i in range(0, len(mat_list)):
    cur_cf_list = pq.bonds.gen_cf(t_lists[i], cpn_rate_list[i], freq_list[i])
    cf_lists.append(cur_cf_list)
cf_lists

[array([   0. ,    2.5,  102.5]),
 array([   0.,    0.,    0.,  100.]),
 array([   0.,    4.,    4.,    4.,    4.,    4.,    4.,    4.,  104.])]

### Specialized functions

In [5]:
def lin_interp(x, x1, x2, y1, y2):
    assert x2 != x1
    slope = (y2 - y1) / (x2 - x1)
    if slope == 0:
        return y1
    else:
        return y1 + slope * (x - x1)

In [12]:
def specialized_newton(x_guess, 
                       f_of_x_dynamic, 
                       fprime_of_x_dynamic,
                       known_rates, 
                       unknown_t_list, 
                       t_list, 
                       cf_of_t, 
                       price,
                       tol_consec=10**-6, 
                       max_iter=100,
                       is_verbose=False):
    cur_iter = 0
    cur_x = x_guess

    cur_f_val = f_of_x_dynamic(cur_x, known_rates, unknown_t_list, t_list, cf_of_t, price)
    cur_chg = cur_f_val
    if is_verbose: print("f(initial guess) =", cur_f_val)

    while cur_iter < max_iter and np.abs(cur_chg) > tol_consec:
        if is_verbose: 
            print("Not close enough, doing next iteration: %s" % str(cur_iter + 1))
            print("Current iteration %s" % (cur_iter+1))
        cur_deriv = fprime_of_x_dynamic(cur_x, known_rates, unknown_t_list, t_list, cf_of_t)
        cur_x = cur_x - (cur_f_val / cur_deriv)
        if is_verbose: 
                print("new x =", cur_x)
        prev_f_val = cur_f_val
        cur_f_val = f_of_x_dynamic(cur_x, known_rates, unknown_t_list, t_list, cf_of_t, price)
        if is_verbose: 
                print("new f(x) =", cur_f_val)
        cur_chg = (cur_f_val - prev_f_val)
        if is_verbose: 
            print("f(x) change this iteration =", cur_chg, "\n")
        cur_iter += 1

        if is_verbose: print("zero was found after %s iterations ... " % cur_iter)
    return cur_x


In [7]:
def f_of_x_dynamic(x, known_rates, unknown_t_list, t_list, cf_of_t, price):
    """ returns the npv of the bond when the largest unknown rate = x """
    largest_known_t = np.max(list(known_rates.keys()))  # 1 -> rate = largest_known_rate
    largest_known_rate = known_rates[largest_known_t] 
    largest_unknown_t = np.max(unknown_t_list)  # 3 -> rate = x
    cur_range = largest_unknown_t - largest_known_t  # = 2
    
    # first sum all of the known rates
    npv = 0
    for t in known_rates:
        npv += np.exp(-known_rates[t]*t) * cf_of_t[t]
    for t in unknown_t_list:
        cur_r = ((t - largest_known_t) * x + 
                 (largest_unknown_t - t) * largest_known_rate) / cur_range
        npv += np.exp(-cur_r*t) * cf_of_t[t]
    return npv - price
    
    
def fprime_of_x_dynamic(x, known_rates, unknown_t_list, t_list, cf_of_t):
    """ returns the derive of the npv function """
    largest_known_t = np.max(list(known_rates.keys()))  # 1 -> rate = largest_known_rate
    largest_known_rate = known_rates[largest_known_t] 
    largest_unknown_t = np.max(unknown_t_list)  # 3 -> rate = x
    cur_range = largest_unknown_t - largest_known_t
    
    deriv = 0.0
    for t in unknown_t_list:
        cur_r = ((t - largest_known_t) * x + 
                 (largest_unknown_t - t) * largest_known_rate) / cur_range
        deriv += np.exp(-cur_r*t) * cf_of_t[t] * (-t * (t - largest_known_t) / cur_range)
    return deriv   

In [14]:
def solve_df_system(t_list, cf_list, px, known_rates, tol, newton_guess=0.05):
    cf_of_t = dict(zip(t_list, cf_list))
    unknown_t_list = []
    for t in t_list:
        if t not in known_rates:
            unknown_t_list.append(t)
    
    print("unknown_t_list:", unknown_t_list)
    # easy case of just 1 rates not known
    if len(unknown_t_list) == 1:
        unknown_t = unknown_t_list[0]
        npv = 0.0
        for t in known_rates:
            npv += np.exp(-known_rates[t]*t) * cf_of_t[t]
        resid = px - npv
        df = resid / cf_of_t[unknown_t]
        new_rate = -np.log(df) / unknown_t
        return {unknown_t: new_rate}
    else:  # complicated case of more than 1 rate not known
        largest_known_t = np.max(list(known_rates.keys()))
        largest_known_rate = known_rates[largest_known_t] 
        largest_unknown_t = np.max(unknown_t_list)
        solved_largest_rate = specialized_newton(newton_guess, 
                                                 f_of_x_dynamic, 
                                                 fprime_of_x_dynamic,
                                                 known_rates, 
                                                 unknown_t_list, 
                                                 t_list, 
                                                 cf_of_t, 
                                                 px,
                                                 tol_consec=tol,
                                                 is_verbose=True)
        rt_dict = {}
        for unknown_t in unknown_t_list:
            if unknown_t != largest_unknown_t:
                cur_rate = lin_interp(unknown_t, 
                                      largest_known_t, largest_unknown_t,
                                      largest_known_rate, solved_largest_rate)
                rt_dict[unknown_t] = cur_rate
        rt_dict[largest_unknown_t] = solved_largest_rate
        return rt_dict
        
        
def bootstrap_dfs(t_lists, cf_lists, px_list, known_rates, tol_list, newton_guess=0.05):
    """ assumes linear interp between r(0,t) """
    i = 0
    for t_list in t_lists:
        cur_mat = np.max(t_list)
        print("running solve_df_system for:\n- t_list %s\n- cf_list "
              "%s\n- px %s\n- known_rates %s" 
              % (t_list, cf_lists[i], px_list[i], known_rates))
        new_rates = solve_df_system(t_list, cf_lists[i], px_list[i], 
                                    known_rates, tol_list[i], newton_guess)
        print("solved rates: %s" % new_rates)
        known_rates.update(new_rates)
        print("updated known_rates:")
        for curkey in known_rates:
            print(curkey, ":", known_rates[curkey])
        
        print("\n")
        i += 1

In [15]:
known_rates = init_rates.copy()
tol_list = [10 ** -6] * len(t_lists)
tol_list

[1e-06, 1e-06, 1e-06]

In [16]:
bootstrap_dfs(t_lists, cf_lists, px_list, known_rates, tol_list, 0.05)

running solve_df_system for:
- t_list [ 0.   0.5  1. ]
- cf_list [   0.     2.5  102.5]
- px 103.0
- known_rates {0: 0.015}
unknown_t_list: [0.5, 1.0]
f(initial guess) = -3.03928069124
Not close enough, doing next iteration: 1
Current iteration 1
new x = 0.0190235793045
new f(x) = 0.0473387111998
f(x) change this iteration = 3.08661940243 

zero was found after 1 iterations ... 
Not close enough, doing next iteration: 2
Current iteration 2
new x = 0.019491407589
new f(x) = 1.10206157302e-05
f(x) change this iteration = -0.0473276905841 

zero was found after 2 iterations ... 
Not close enough, doing next iteration: 3
Current iteration 3
new x = 0.0194915165518
new f(x) = 6.11066752754e-13
f(x) change this iteration = -1.10206151192e-05 

zero was found after 3 iterations ... 
Not close enough, doing next iteration: 4
Current iteration 4
new x = 0.0194915165518
new f(x) = 0.0
f(x) change this iteration = -6.11066752754e-13 

zero was found after 4 iterations ... 
solved rates: {0.5: 0.0

In [6]:
pq.interpolate.lin_interp(3.5, 1.5, 4.0, 0.0237514517621, 0.0373664841487)

0.03464347767138